In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder.appName("BigData").getOrCreate()

In [3]:
spark

# Importando datos con PySpark

In [4]:
df = spark.read.csv("telco_customer_churn.csv", 
                    header=True,
                    inferSchema=True)

In [257]:
var_cat = [nC for nC, dt in df.dtypes if dt == "string"]

var_num = [nC for nC, dt in df.dtypes if dt in ["int", "double"]]

In [6]:
df.select(var_num).limit(5).toPandas()

,SeniorCitizen,tenure,MonthlyCharges
0,0,1,29.85
1,0,34,56.95
2,0,2,53.85
3,0,45,42.30
4,0,2,70.70


In [96]:
df.select(var_num).describe().toPandas()

,summary,SeniorCitizen,tenure,MonthlyCharges
0,count,7043,7043,7043
1,mean,0.1621468124378816,32.37114865824223,64.76169246059922
2,stddev,0.3686116056100135,24.559481023094442,30.090047097678482
3,min,0,0,18.25
4,max,1,72,118.75


In [97]:
df.select(var_cat).limit(5).toPandas()

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
0,7590-VHVEG,Female,Yes,No,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,No
1,5575-GNVDE,Male,No,No,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,1889.5,No
2,3668-QPYBK,Male,No,No,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,108.15,Yes
3,7795-CFOCW,Male,No,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),1840.75,No
4,9237-HQITU,Female,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,151.65,Yes


In [98]:
# ID
var_cat.remove("customerID")
# Variable respuesta
var_cat.remove("Churn")
# Variable numerica
var_cat.remove("TotalCharges")

In [74]:
#var_num.append("TotalCharges")

In [99]:
var_cat

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [116]:
var_num

['SeniorCitizen', 'tenure', 'MonthlyCharges']

In [101]:
lista_etapas = []
for cat in var_cat:
    strIdx = StringIndexer(inputCol=cat, outputCol=cat+'_index')
    encoder = OneHotEncoder(inputCol=cat+'_index', outputCol=cat+'_oneHot')
    
    lista_etapas += [strIdx, encoder]

In [102]:
lista_etapas

[StringIndexer_7803163ccf6c,
 OneHotEncoder_e8021b2ceac9,
 StringIndexer_3528287260b0,
 OneHotEncoder_279a415afd01,
 StringIndexer_d7baf37dfb34,
 OneHotEncoder_f1aad387d1f7,
 StringIndexer_e4bee6b58a1c,
 OneHotEncoder_2305ca532042,
 StringIndexer_6b6f85ede9e1,
 OneHotEncoder_94dd73729f5d,
 StringIndexer_9ee0f059279a,
 OneHotEncoder_ce0fd8c3e5db,
 StringIndexer_53a77f2cd4fb,
 OneHotEncoder_8f5e68d8591d,
 StringIndexer_70f42266ab78,
 OneHotEncoder_4eb482794af4,
 StringIndexer_b30f59a97274,
 OneHotEncoder_532f8b5d51d8,
 StringIndexer_63cfbb27346c,
 OneHotEncoder_db137f4f9014,
 StringIndexer_7a31ef4129c0,
 OneHotEncoder_d00b74cf3356,
 StringIndexer_9080ac7a8951,
 OneHotEncoder_462e5742d126,
 StringIndexer_25b3b4f79d71,
 OneHotEncoder_a5b16758ba1b,
 StringIndexer_7c63363009d1,
 OneHotEncoder_5d67c93c60d0,
 StringIndexer_785efdd463e1,
 OneHotEncoder_df24d28c2345]

In [103]:
len(lista_etapas)

30

In [104]:
lista_etapas[0].getInputCol()

'gender'

In [81]:
lista_etapas[0].getOutputCol()

'gender_index'

In [82]:
lista_etapas[1].getInputCol()

'gender_index'

In [105]:
lista_etapas[1].getOutputCol()

'gender_oneHot'

**Tratemiento para la variable categorica Y [Churn]**

In [106]:
strIdx2 = StringIndexer(inputCol="Churn", outputCol="Y")
lista_etapas.append(strIdx2)

In [107]:
lista_etapas

[StringIndexer_7803163ccf6c,
 OneHotEncoder_e8021b2ceac9,
 StringIndexer_3528287260b0,
 OneHotEncoder_279a415afd01,
 StringIndexer_d7baf37dfb34,
 OneHotEncoder_f1aad387d1f7,
 StringIndexer_e4bee6b58a1c,
 OneHotEncoder_2305ca532042,
 StringIndexer_6b6f85ede9e1,
 OneHotEncoder_94dd73729f5d,
 StringIndexer_9ee0f059279a,
 OneHotEncoder_ce0fd8c3e5db,
 StringIndexer_53a77f2cd4fb,
 OneHotEncoder_8f5e68d8591d,
 StringIndexer_70f42266ab78,
 OneHotEncoder_4eb482794af4,
 StringIndexer_b30f59a97274,
 OneHotEncoder_532f8b5d51d8,
 StringIndexer_63cfbb27346c,
 OneHotEncoder_db137f4f9014,
 StringIndexer_7a31ef4129c0,
 OneHotEncoder_d00b74cf3356,
 StringIndexer_9080ac7a8951,
 OneHotEncoder_462e5742d126,
 StringIndexer_25b3b4f79d71,
 OneHotEncoder_a5b16758ba1b,
 StringIndexer_7c63363009d1,
 OneHotEncoder_5d67c93c60d0,
 StringIndexer_785efdd463e1,
 OneHotEncoder_df24d28c2345,
 StringIndexer_af7fe4b4a599]

In [108]:
lista_etapas[-1].getInputCol()

'Churn'

**Uniendo los vectores one-hot de las variables cat + variables num**

In [109]:
columnasVectores = [c+'_oneHot' for c in var_cat] + var_num

ensamblador = VectorAssembler(inputCols=columnasVectores, outputCol="X")

lista_etapas.append(ensamblador)

In [110]:
lista_etapas

[StringIndexer_7803163ccf6c,
 OneHotEncoder_e8021b2ceac9,
 StringIndexer_3528287260b0,
 OneHotEncoder_279a415afd01,
 StringIndexer_d7baf37dfb34,
 OneHotEncoder_f1aad387d1f7,
 StringIndexer_e4bee6b58a1c,
 OneHotEncoder_2305ca532042,
 StringIndexer_6b6f85ede9e1,
 OneHotEncoder_94dd73729f5d,
 StringIndexer_9ee0f059279a,
 OneHotEncoder_ce0fd8c3e5db,
 StringIndexer_53a77f2cd4fb,
 OneHotEncoder_8f5e68d8591d,
 StringIndexer_70f42266ab78,
 OneHotEncoder_4eb482794af4,
 StringIndexer_b30f59a97274,
 OneHotEncoder_532f8b5d51d8,
 StringIndexer_63cfbb27346c,
 OneHotEncoder_db137f4f9014,
 StringIndexer_7a31ef4129c0,
 OneHotEncoder_d00b74cf3356,
 StringIndexer_9080ac7a8951,
 OneHotEncoder_462e5742d126,
 StringIndexer_25b3b4f79d71,
 OneHotEncoder_a5b16758ba1b,
 StringIndexer_7c63363009d1,
 OneHotEncoder_5d67c93c60d0,
 StringIndexer_785efdd463e1,
 OneHotEncoder_df24d28c2345,
 StringIndexer_af7fe4b4a599,
 VectorAssembler_f5563e3a0002]

**Aplicando las etapas al dataset - Pipeline**

In [111]:
procesadorEtapas = Pipeline(stages=lista_etapas)

In [112]:
modelo = procesadorEtapas.fit(df)

In [113]:
df2 = modelo.transform(df)

In [115]:
df2.limit(5).toPandas().transpose()

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [117]:
df2.select(["X", "Y"]).limit(5).toPandas()

,X,Y
0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...",0.0
1,"(1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...",0.0
2,"(1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...",1.0
3,"(1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0.0
4,"(0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...",1.0


In [118]:
df.limit(5).toPandas()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Ejercicios:
    1. Transformar la variable TotalCharge en una variable num
    2. Construir un nuevo ensamblador considerando la nueva variable 
    3. Normalizar las variables numericas entre valores 0-1
    4. Finalmente, construir el ensamblador considerando los datos normalizados

### Transformando TotalCharges a num 

In [139]:
df_telco = df.withColumn("TotalCharges", df["TotalCharges"].cast("double"))

In [140]:
df_telco.select("TotalCharges").dtypes

[('TotalCharges', 'double')]

In [141]:
df_telco.select("TotalCharges").describe().toPandas()

,summary,TotalCharges
0,count,7032
1,mean,2283.3004408418697
2,stddev,2266.771361883145
3,min,18.8
4,max,8684.8


### Nuevo ensamblador

In [142]:
var_cat = [nC for nC, dt in df_telco.dtypes if dt == "string"]
var_num = [nC for nC, dt in df_telco.dtypes if dt in ["int", "double"]]

In [143]:
var_num

['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']

In [144]:
var_cat.remove("customerID")
# Variable respuesta
var_cat.remove("Churn")

In [145]:
lista_etapas = []
for cat in var_cat:
    strIdx = StringIndexer(inputCol=cat, outputCol=cat+'_index')
    encoder = OneHotEncoder(inputCol=cat+'_index', outputCol=cat+'_oneHot')
    
    lista_etapas += [strIdx, encoder]

In [146]:
strIdx2 = StringIndexer(inputCol="Churn", outputCol="Y")
lista_etapas.append(strIdx2)

In [147]:
columnasVectores = [c+'_oneHot' for c in var_cat] + var_num

ensamblador = VectorAssembler(inputCols=columnasVectores, outputCol="X")

lista_etapas.append(ensamblador)

In [148]:
procesadorEtapas = Pipeline(stages=lista_etapas)

In [149]:
modelo = procesadorEtapas.fit(df_telco)

In [150]:
df_telco2 = modelo.transform(df_telco)

In [151]:
df_telco2.limit(5).toPandas().transpose()

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


### Normalizacion

In [128]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.functions import vector_to_array

In [129]:
df_telco = df.withColumn("TotalCharges", df["TotalCharges"].cast("double"))

In [105]:
var_num

['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']

In [130]:
for col in var_num:
    null_qty = df_telco.filter(df_telco[col].isNull()).count()
    print(col, null_qty)

SeniorCitizen 0
tenure 0
MonthlyCharges 0
TotalCharges 11


In [131]:
df_telco = df_telco.na.drop(subset=["TotalCharges"])

In [132]:
for col in var_num:
    null_qty = df_telco.filter(df_telco[col].isNull()).count()
    print(col, null_qty)

SeniorCitizen 0
tenure 0
MonthlyCharges 0
TotalCharges 0


In [133]:
stages_list = []
for col in var_num:
    assembler = VectorAssembler(inputCols=[col], outputCol=col+"_vec")
    normalizer = MinMaxScaler(inputCol=col+"_vec", outputCol=col+"_norm")
    
    stages_list += [assembler, normalizer]
    
pipeline = Pipeline(stages=stages_list)
model = pipeline.fit(df_telco)
df_telco_norm = model.transform(df_telco)

In [134]:
for col in var_num:
    df_telco_norm = df_telco_norm.drop(col+"_vec")

In [135]:
for col in var_num:
    df_telco_norm = df_telco_norm.withColumn(col, vector_to_array(col+"_norm"))

In [136]:
for col in var_num:
    df_telco_norm = df_telco_norm.drop(col+"_norm")

In [137]:
for col in var_num:
    df_telco_norm = df_telco_norm.withColumn(col, df_telco_norm[col].getItem(0))

In [138]:
df_telco_norm.select(var_num).limit(10).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9
SeniorCitizen,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
tenure,0.000000,0.464789,0.014085,0.619718,0.014085,0.098592,0.295775,0.126761,0.380282,0.859155
MonthlyCharges,0.115423,0.385075,0.354229,0.239303,0.521891,0.809950,0.704975,0.114428,0.861194,0.377114
TotalCharges,0.001275,0.215867,0.010310,0.210241,0.015330,0.092511,0.222779,0.032668,0.349325,0.400317


### Nuevo ensamblador con los datos normalizados

In [118]:
var_cat.remove("customerID")
# Variable respuesta
var_cat.remove("Churn")

In [119]:
lista_etapas = []
for cat in var_cat:
    strIdx = StringIndexer(inputCol=cat, outputCol=cat+'_index')
    encoder = OneHotEncoder(inputCol=cat+'_index', outputCol=cat+'_oneHot')
    
    lista_etapas += [strIdx, encoder]

strIdx2 = StringIndexer(inputCol="Churn", outputCol="Y")
lista_etapas.append(strIdx2)

In [120]:
columnasVectores = [c+'_oneHot' for c in var_cat] + var_num
ensamblador = VectorAssembler(inputCols=columnasVectores, outputCol="X")
lista_etapas.append(ensamblador)

In [121]:
pipeline = Pipeline(stages=lista_etapas)
df_telco_final = pipeline.fit(df_telco_norm).transform(df_telco_norm)

In [126]:
df_telco_final.select(["X", "Y"]).limit(20).toPandas()

,X,Y
0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...",0.0
1,"(1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...",0.0
2,"(1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...",1.0
3,"(1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0.0
4,"(0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...",1.0
5,"(0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...",1.0
6,"(1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...",0.0
7,"(0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0.0
8,"(0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...",1.0
9,"(1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...",0.0
